In [ ]:
#obtention de la taille de l'investissement F_t (taux en asset par rapport au cash)
#a la manière d'un Single Layer Perceptron, la taille de la position est calculée avec un produit scalaire entre chaque poids de theta et chaque feature (r_t-m, r_t-m+1, r_t-m+2,... , r_t)
#on obtient donc F_t après avoir passé le scalaire dans un tanh pr obtenir une fraction entre -1 et 1 (sell et buy)

#x est le vecteur des rentabilités sur l'ensemble.

def positions(x, theta):
    M = len(theta) - 2
    T = len(x)
    Ft = np.zeros(T)
    for t in range(M, T):
        xt = np.concatenate([[1], x[t - M:t], [Ft[t - 1]]])
        Ft[t] = np.tanh(np.dot(theta, xt))
    return Ft

In [ ]:
#calcul des rentabilités de la stratégie en multipliant les fractions des positions par les rentabilités de l'asset (profit si signe(returns)=signe(fraction) et perte si inverse) du jour même
#delta = taux de commission
def returns(Ft, x, delta):
    T = len(x)
    rets = Ft[0:T - 1] * x[1:T] - delta * np.abs(Ft[1:T] - Ft[0:T - 1])
    return np.concatenate([[0], rets])

In [ ]:
#calcul du gradient de theta, pour update theta avec un poids alpha (learning_rate), S est le sharpe
def gradient(x, theta, delta):
    Ft = positions(x, theta)
    R = returns(Ft, x, delta)
    T = len(x)
    M = len(theta) - 2
    
    A = np.mean(R)
    B = np.mean(np.square(R))
    S = A / np.sqrt(B - A ** 2)

    dSdA = S * (1 + S ** 2) / A
    dSdB = -S ** 3 / 2 / A ** 2
    dAdR = 1. / T
    dBdR = 2. / T * R
    
    grad = np.zeros(M + 2)
    dFpdtheta = np.zeros(M + 2)  
    
    for t in range(M, T):
        xt = np.concatenate([[1], x[t - M:t], [Ft[t-1]]])
        dRdF = -delta * np.sign(Ft[t] - Ft[t-1])
        dRdFp = x[t] + delta * np.sign(Ft[t] - Ft[t-1])
        dFdtheta = (1 - Ft[t] ** 2) * (xt + theta[-1] * dFpdtheta)
        dSdtheta = (dSdA * dAdR + dSdB * dBdR[t]) * (dRdF * dFdtheta + dRdFp * dFpdtheta)
        grad = grad + dSdtheta
        dFpdtheta = dFdtheta

        
    return grad, S

In [ ]:
def train(x, epochs=2000, M=8, commission=0.0025, learning_rate = 0.3):
    theta = np.random.rand(M + 2) #initialisation aléatoire des poids
    sharpes = np.zeros(epochs) #stockage des sharpe
    for i in range(epochs):
        grad, sharpe = gradient(x, theta, commission)
        theta = theta + grad * learning_rate

        sharpes[i] = sharpe
    
    
    print("finished training")
    return theta, sharpes